In [ ]:
import numpy as np
import pandas as pd
import os

import chemprop

from sklearn.metrics import confusion_matrix

In [ ]:
np.random.seed(1234)

In [ ]:
endpoint = 'skin-sensitization'
# endpoint = 'eye-irritation'

loc = r'D:\School\Semester3\Seminar - Reproducibility\seminar-toxicity\data'
endpoint_loc = os.path.join(loc, endpoint)
model = r'D:\School\Semester3\Seminar - Reproducibility\seminar-toxicity\src\models'
model_loc = os.path.join(model, endpoint)

In [ ]:
# train_filename = 'train.csv'
# val_filename = 'val.csv'
# arguments = [
#     '--data_path', os.path.join(endpoint_loc, train_filename),
#     '--dataset_type', 'classification',
#     '--config_path', os.path.join(model_loc, 'config.json'),
#     '--separate_test_path', os.path.join(endpoint_loc, val_filename), 
#     '--save_dir', model_loc,
#     '--target_columns', 'Activity', 
#     '--smiles_columns', 'SMILES',
#     '--features_generator', 'rdkit_2d_normalized', 
#     '--no_features_scaling', 
#     '--ensemble_size', '5', 
#     '--extra_metrics', 'accuracy'
# ]

# args = chemprop.args.TrainArgs().parse_args(arguments)
# # chemprop.train.run_training(args=args)
# mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

In [ ]:
val_filename = 'val.csv'
arguments = [
    # '--test_path', os.path.join(endpoint_loc, val_filename),
    '--test_path', '/dev/null', 
    '--preds_path', '/dev/null',
    '--checkpoint_dir', model_loc,
    '--smiles_columns', 'SMILES',
    '--features_generator', 'rdkit_2d_normalized', 
    '--no_features_scaling'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args, smiles=smiles_input)

In [ ]:
smiles_input = []
for smiles in val_smiles:
    smiles_input.append([smiles])

In [ ]:
filename = 'train.csv'
df_train = pd.read_csv(os.path.join(endpoint_loc, filename))

In [ ]:
df_train.shape

In [ ]:
df_train.head()

In [ ]:
filename = 'val.csv'
df_val = pd.read_csv(os.path.join(endpoint_loc, filename))

In [ ]:
df_val.shape

In [ ]:
df_val.head()

In [ ]:
train_smiles = df_train['SMILES'].to_numpy()
train_labels = df_train['Activity'].to_numpy()

val_smiles = df_val['SMILES'].to_numpy()
val_labels = df_val['Activity'].to_numpy()

In [ ]:
print('val size smiles :', val_smiles.shape)
print('val size labels :', val_labels.shape)
print('pos samples in val size :', val_labels[val_labels == 1].shape)
print('neg samples in val size :', val_labels[val_labels == 0].shape)

In [ ]:
y_pred = (np.array(preds).flatten()[np.where(np.array(preds).flatten() != 'Invalid SMILES')].astype(np.float32) > 0.5).astype(np.int64)

In [ ]:
y_true = val_labels[np.where(np.array(preds).flatten() != 'Invalid SMILES')]

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_pred, y_true).ravel()

ACC = (tp + tn)/(tp + tn + fn + fp)
SEN = tp/(tp + fn)
SPE = tn/(tn + fp)

In [ ]:
print(f'Accuracy = {ACC}')
print(f'Sensitivity = {SEN}')
print(f'Specificity = {SPE}')